In [2]:
from selenium import webdriver
import time
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
from urllib.parse import urljoin
import re
import time

In [3]:
driver =  webdriver.Chrome(executable_path = 'D:\workspace\chromedriver.exe' )

driver.get('https://ev.or.kr/evmonitor')
#time.sleep(5)

In [4]:
count=0
if count==0:
    tmp_len=0
    create_date=[]

In [5]:
sid = []

In [6]:
for k in range(2,19):
    driver.find_element_by_css_selector('#F_SI_DO1')
    driver.implicitly_wait(5)
    driver.find_element_by_css_selector('#F_SI_DO1 > option:nth-child(%s)'%k).click()
    #time.sleep(3)
    driver.find_element_by_css_selector('#statList')
    driver.implicitly_wait(30)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    a = soup.find_all('div',class_='fw_path')
    for link in a:
        temp = str(link['sid'])
        sid.append(temp)


In [7]:
len(sid)

2492

In [8]:
tmp_len2 = len(sid)-tmp_len
    
if tmp_len != len(sid):
    tmp_len2 = len(sid)-tmp_len
for i in range (0,tmp_len2):
    i = time.ctime()
    create_date.append(i)
    
tmp_len = len(sid)
count+=1

In [9]:
print(len(create_date))

2492


In [10]:
#충전소_딕셔너리
c_station = {'name':[], 'u_time':[], 'agency': [], 'city':[], 'gu':[], 'add':[],'phone':[], 'fee':[],'etc':[]}
#충전기_딕셔너리
charger = {'c_name':[], 'typ':[], 'charge':[], 'using':[], 'amount':[]}

In [11]:
count2=0
if count2==0:
    tmp_len_2=0
    c_create_date=[]

In [12]:
for i in sid[0:10]:
    url_base = 'https://www.ev.or.kr/portal/monitor/evminfo?sid='
    url = url_base+str(i)
    html = urlopen(url)
    soup = BeautifulSoup(html, 'html.parser')
    
    for tmp_using in soup.find_all(class_='table_01'):
        #if len(tmp_using)<=5:
        n_tmp = soup.find_all('h2')
        c_station['name'].append(n_tmp[0].get_text())

        t_tmp = soup.find_all('em')
        c_station['u_time'].append(t_tmp[0].get_text())

        info_tmp = soup.find_all(class_='table_02')
        dinfo = info_tmp[0].find_all('td')
        c_station['add'].append(dinfo[0].get_text())
        c_station['agency'].append(dinfo[1].get_text())
        c_station['phone'].append(dinfo[2].get_text())
    
        tmp_fee = dinfo[3].get_text()
        tmp_fee = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_fee)
        c_station['fee'].append(tmp_fee)
        
            
        tmp_etc = dinfo[5].get_text()
        tmp_etc = re.sub('&nbsp; | &nbsp; |\n|\t|\r|', '', tmp_etc)
        c_station['etc'].append(tmp_etc)
        
        city_tmp = dinfo[0].get_text()[0:2]
        gu_tmp = dinfo[0].get_text().split(' ')
        gu_tmp = gu_tmp[1]
        c_station['gu'].append(gu_tmp)
        c_station['city'].append(city_tmp)
        
        if len(tmp_using)<=5:
            
            n_tmp = soup.find_all('h2')
            charger['c_name'].append(n_tmp[0].get_text())

            ty_tmp = soup.find_all('dd')
            charger['typ'].append(ty_tmp[0].get_text())
            
            tmp_amount = dinfo[4].get_text()
            
            tmp_amount = re.findall('\d+', tmp_amount) #숫자만 추출
            charger['amount'].append(tmp_amount)
            
            for u_tmp in soup.find_all(class_='td3'):
                tmp = re.sub('\n', '', u_tmp.get_text())
                charger['using'].append(tmp)
            
    
            td2 = soup.find_all(class_='td2')
            for te in td2[0:]:
                test=[]
                for a in te.find_all('span'):
                    if a.get_text()=='사용가능':
                        break
                    elif a.get_text()=='사용중':
                        break
                    elif a.get_text()=='사용불가':
                        break
                    elif a.get_text()==' ':
                        break
                    elif len(a.get_text())<6:
                        tmp=[]
                        tmp.append(a.get_text())
                        #print(tmp)
                        test+=tmp
                test= str("+".join(test))
                charger['charge'].append(test)
                if test!=None:
                    break

                
            
        elif len(tmp_using)<=7:
            tmp_using = soup.find_all(class_='ev_type t03')
                #print(tmp_using)
            for te in tmp_using[0:]:
                    #print(te)
                for a in te.find_all('span'):
                    if a.get_text()==' ':
                        break
                    elif len(a.get_text())<6:
                        charger['charge'].append(a.get_text())

                        n_tmp = soup.find_all('h2')
                        charger['c_name'].append(n_tmp[0].get_text())

                        ty_tmp = soup.find_all('dd')
                        charger['typ'].append(ty_tmp[0].get_text())

                        tmp_amount = dinfo[4].get_text()
            
                        tmp_amount = re.findall('\d+', tmp_amount) #숫자만 추출
                        charger['amount'].append(tmp_amount)

                        for u_tmp in soup.find_all(class_='td3'):
                            tmp = re.sub('\n', '', u_tmp.get_text())
                            charger['using'].append(tmp)
                            break


In [13]:
import time

In [15]:
tmp_len3 = len(charger['c_name'])-tmp_len_2
    
if tmp_len3 != len(charger['c_name']):
    tmp_len3 = len(charger['c_name'])-tmp_len_2
for i in range (0,tmp_len3):
    i = time.ctime()
    c_create_date.append(i)

tmp_len3 = len(charger['c_name'])
count2+=1

In [17]:
print(len(c_create_date), len(charger['c_name']))

14 14


In [19]:
len(c_station['name'])

10

In [21]:
df = pd.DataFrame({'이름':c_station['name'], '사용가능시간':c_station['u_time'], '운영기관':c_station['agency'],'도시':c_station['city'], '구': c_station['gu'],'도로명주소': c_station['add'],'연락처': c_station['phone'],'충전요금': c_station['fee'], '참고사항': c_station['etc']})
df

이름       사용가능시간            운영기관  도시    구  \
0        DMC 산학협력연구센터    24시간 이용가능  환경부(한국자동차환경협회)  서울  마포구   
1     Park M (주차복합빌딩)    24시간 이용가능  환경부(한국자동차환경협회)  서울  마포구   
2               SETEC    24시간 이용가능  환경부(한국자동차환경협회)  서울  강남구   
3          sh서울주택도시공사    24시간 이용가능  환경부(한국자동차환경협회)  서울  강남구   
4       가로공원로 지하공영주차장    24시간 이용가능  환경부(한국자동차환경협회)  서울  강서구   
5             가양레포츠센터    24시간 이용가능  환경부(한국자동차환경협회)  서울  강서구   
6           가오리 공영주차장    24시간 이용가능  환경부(한국자동차환경협회)  서울  강북구   
7  강남구청 삼성로 별관 부설 주차장  09:00~18:00  환경부(한국자동차환경협회)  서울  강남구   
8    강남치매지원센터 공영노외주차장    24시간 이용가능  환경부(한국자동차환경협회)  서울  강남구   
9                강동구청    24시간 이용가능  환경부(한국자동차환경협회)  서울  강동구   

                           도로명주소        연락처 충전요금  \
0             서울특별시 마포구 매봉산로 37   1661-9408   유료   
1             서울특별시 마포구 매봉산로 80   1661-9408   무료   
2           서울특별시 강남구 남부순환로3104   1661-9408   유료   
3             서울특별시 강남구 개포로 621   1661-9408   유료   
4  서울특별시 강서구 가로공원로 189  지하주차장 05  1661-9408   유료   
5          서울특별시 강서구 양천로61길 101   1661-9408   유료   
6          서울특별시 강북구 삼양로107길 30   1661-9408   유료   
7             서울특별시 강남구 삼성로 628   1661-9408   무료   
8          서울특별시 강남구 선릉로108길 27   1661-9408   무료   
9              서울특별시 강동구 성내로 25   1661-9408   유료   

                                    참고사항  
0                              10분/500원   
1  최초30분 1500원, 10분당 500원 주민센터이용 50% 할인   
2    충전차량에한하여 1시간무료 10분당500월 일주차15,000원   
3                                    없음   
4                               주차요금 별도   
5                                    없음   
6                   5분/150원 친환경차 50% 감면   
7                       5분당 300원, 주말 무료   
8                               5분당400원   
9                   5분 250원 친환경차 50% 감면

In [22]:
c_df = pd.DataFrame({'이름':charger['c_name'], '구분':charger['typ'], '충전기타입':charger['charge'], '사용중':charger['using'], '급속충전량':charger['amount']})
c_df

이름  구분            충전기타입   사용중  급속충전량
0         DMC 산학협력연구센터  급속  DC차데모+AC3상+DC콤보  사용가능  [100]
1      Park M (주차복합빌딩)  급속             DC콤보  사용가능  [200]
2      Park M (주차복합빌딩)  급속             DC콤보  사용가능  [200]
3                SETEC  급속  DC차데모+AC3상+DC콤보  사용가능  [100]
4           sh서울주택도시공사  급속             DC콤보   사용중  [200]
5           sh서울주택도시공사  급속             DC콤보   사용중  [200]
6        가로공원로 지하공영주차장  급속  DC차데모+AC3상+DC콤보  사용가능   [50]
7              가양레포츠센터  급속  DC차데모+AC3상+DC콤보  사용가능  [100]
8            가오리 공영주차장  급속  DC차데모+AC3상+DC콤보  사용가능  [100]
9   강남구청 삼성로 별관 부설 주차장  급속             DC콤보   사용중  [100]
10    강남치매지원센터 공영노외주차장  급속             DC콤보  사용가능  [200]
11    강남치매지원센터 공영노외주차장  급속             DC콤보  사용가능  [200]
12                강동구청  급속             DC콤보  사용가능  [200]
13                강동구청  급속             DC콤보  사용가능  [200]